In [ ]:
ws_url= 'wss://api.us-south.speech-to-text.watson.cloud.ibm.com/instances/so4OG0zBh-S0pbvPQtt3TuS1mysBGAdx57rBKMTZLYXl/v1/recognize'

In [ ]:
#import ibm watson and os
import os
!pip install ibm_watson
#import other ibm dependancies
import json
from os.path import join,dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException

In [ ]:
apikey=''
authenticator = IAMAuthenticator(apikey)
print(authenticator)

In [ ]:
!curl -k -X POST \
  --header "Content-Type: application/x-www-form-urlencoded" \
  --header "Accept: application/json" \
  --data-urlencode "grant_type=urn:ibm:params:oauth:grant-type:apikey" \
  --data-urlencode "apikey=" \
  "https://iam.cloud.ibm.com/identity/token"

{"access_token":"eyJraWQiOiIyMDIwMTIyMTE4MzQiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJpYW0tU2VydmljZUlkLTViY2I3MjBhLTFlY2ItNGQ1Ny1hYzliLTNlZDdmZWI2NzEyYiIsImlkIjoiaWFtLVNlcnZpY2VJZC01YmNiNzIwYS0xZWNiLTRkNTctYWM5Yi0zZWQ3ZmViNjcxMmIiLCJyZWFsbWlkIjoiaWFtIiwianRpIjoiYjVhYmYyYzAtNDE0Mi00NjQyLWE1NTYtNWYyYzMwYTUxNDUzIiwiaWRlbnRpZmllciI6IlNlcnZpY2VJZC01YmNiNzIwYS0xZWNiLTRkNTctYWM5Yi0zZWQ3ZmViNjcxMmIiLCJuYW1lIjoiQXV0by1nZW5lcmF0ZWQgc2VydmljZSBjcmVkZW50aWFscyIsInN1YiI6IlNlcnZpY2VJZC01YmNiNzIwYS0xZWNiLTRkNTctYWM5Yi0zZWQ3ZmViNjcxMmIiLCJzdWJfdHlwZSI6IlNlcnZpY2VJZCIsInVuaXF1ZV9pbnN0YW5jZV9jcm5zIjpbImNybjp2MTpibHVlbWl4OnB1YmxpYzpzcGVlY2gtdG8tdGV4dDp1cy1zb3V0aDphL2RhY2Q5ODk3NTU3MjQ3ZDVhYzgxZjdjZTViZTZkODZhOjNhNjc3YjJiLWMwMmQtNDRkMS1hNWJjLWQwZmZiYjVmYzBjYTo6Il0sImFjY291bnQiOnsidmFsaWQiOnRydWUsImJzcyI6ImRhY2Q5ODk3NTU3MjQ3ZDVhYzgxZjdjZTViZTZkODZhIiwiZnJvemVuIjp0cnVlfSwiaWF0IjoxNjEwNzgwMDM1LCJleHAiOjE2MTA3ODM2MzUsImlzcyI6Imh0dHBzOi8vaWFtLmNsb3VkLmlibS5jb20vaWRlbnRpdHkiLCJncmFudF90eXBlIjoidXJuOmlibTpwYXJhbXM6b2

'eyJraWQiOiIyMDIwMTIyMTE4MzQiLCJhbGciOiJSUzI1NiJ9.eyJpYW1faWQiOiJpYW0tU2VydmljZUlkLTViY2I3MjBhLTFlY2ItNGQ1Ny1hYzliLTNlZDdmZWI2NzEyYiIsImlkIjoiaWFtLVNlcnZpY2VJZC01YmNiNzIwYS0xZWNiLTRkNTctYWM5Yi0zZWQ3ZmViNjcxMmIiLCJyZWFsbWlkIjoiaWFtIiwianRpIjoiOWY0YTM2ZTAtYmM2MC00NTRiLTljMzEtNDczYmQ4NGI0MDRjIiwiaWRlbnRpZmllciI6IlNlcnZpY2VJZC01YmNiNzIwYS0xZWNiLTRkNTctYWM5Yi0zZWQ3ZmViNjcxMmIiLCJuYW1lIjoiQXV0by1nZW5lcmF0ZWQgc2VydmljZSBjcmVkZW50aWFscyIsInN1YiI6IlNlcnZpY2VJZC01YmNiNzIwYS0xZWNiLTRkNTctYWM5Yi0zZWQ3ZmViNjcxMmIiLCJzdWJfdHlwZSI6IlNlcnZpY2VJZCIsInVuaXF1ZV9pbnN0YW5jZV9jcm5zIjpbImNybjp2MTpibHVlbWl4OnB1YmxpYzpzcGVlY2gtdG8tdGV4dDp1cy1zb3V0aDphL2RhY2Q5ODk3NTU3MjQ3ZDVhYzgxZjdjZTViZTZkODZhOjNhNjc3YjJiLWMwMmQtNDRkMS1hNWJjLWQwZmZiYjVmYzBjYTo6Il0sImFjY291bnQiOnsidmFsaWQiOnRydWUsImJzcyI6ImRhY2Q5ODk3NTU3MjQ3ZDVhYzgxZjdjZTViZTZkODZhIiwiZnJvemVuIjp0cnVlfSwiaWF0IjoxNjEwNzc0NDc2LCJleHAiOjE2MTA3NzgwNzYsImlzcyI6Imh0dHBzOi8vaWFtLmNsb3VkLmlibS5jb20vaWRlbnRpdHkiLCJncmFudF90eXBlIjoidXJuOmlibTpwYXJhbXM6b2F1dGg6Z3JhbnQtdH

In [ ]:
var IAM_access_token = IAM_token;
var wsURI = ws_url
  + '?access_token=' + IAM_access_token
  + '&model=en-US_BroadbandModel';
var websocket = new WebSocket(wsURI);

websocket.onopen = function(evt) { onOpen(evt) };
websocket.onclose = function(evt) { onClose(evt) };
websocket.onmessage = function(evt) { onMessage(evt) };
websocket.onerror = function(evt) { onError(evt) };


SyntaxError: ignored

In [ ]:
# to get authenticator token
def transcriber_token(self):
        resp = requests.post(
            "https://iam.cloud.ibm.com/identity/token",
            headers={"Accept": "application/json"},
            params={
                "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
                "apikey": apikey,
            },
        )
        return resp.json()["access_token"]

In [ ]:
def initialize(self, **kwargs):
        self.transcriber = tornado.websocket.websocket_connect(
            on_message_callback=self.on_transcriber_message,
        )